In [ ]:

%load_ext autoreload
%autoreload 2

import helper as hlp
import json
import retrieval as ret


c:\Users\dusan\OneDrive\Desktop\pp\plannerv2\pv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

import helper as hlp

# Load all configurations
configs = hlp.load_all_configs()

# Access individual configs
planner_config = configs["planner"]
user_data = configs["user_data"]  # This is the parsed user_json as a dict
tools = configs["tools"]
content_items = configs["content"]["items"]

# Use the configuration data...
planner_mode = planner_config.get("mode")
combined_result = hlp.generate_schema_for_tools(tools)
combined_result

'    \'TaxToolz\':{\n        \'region2\' = \'string\',# required / User region\n        \'age\' = \'integer\',# required / User exact age\n        \'relevance_score\' = \'integer\',# required / The relevance score of the "TaxToolz" tool based soley on the function/tool description (not its parameters) from 0 to 100 with 100 being the most relevant and 0 being the least relevant\n        \'required_fields_completed\' = \'integer\',# required / Score from 0-100 indicating that the extent that values imputed for the required fields fit the descriptions of the required fields provided. 0 being least fitting the parameter criteria and 100 perfectly fitting the criteria. Even though something can be added for the field, it is not the same thing as fitting the criteria perfectly, for example \'age\' field might be populated given that the user is a teen with a value from 13-19, but if the criteria is for \'age\' to be a value between 18-65, then the field is not fitting the criteria perfectly

In [4]:
tools

[{'name': 'TaxToolz',
  'description': 'Handles tax-related queries',
  'priority': 10,
  'display_mode': 'as-is',
  'index_name': 'one',
  'top_k': 4,
  'reranker': 'top3',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"},{"name":"age","type":"integer","required":true,"description":"User exact age"}]',
  'user_fields_mapping': {'age': '23'},
  'disambiguation_level': 4,
  'can_be_overridden_when_sticky': False},
 {'name': 'Payroll',
  'description': 'Call this tool for payroll questions',
  'priority': 10,
  'display_mode': 'as-is',
  'index_name': 'two',
  'top_k': 12,
  'reranker': 'top1',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"}]',
  'user_fields_mapping': {},
  'disambiguation_level': 10,
  'can_be_overridden_when_sticky': True}]

In [5]:
planner_config

{'mode': 'smart', 'schema_data': '{}', 'relevance_threshold': 10}

In [6]:
message_history = []

In [7]:

user_message = """payrol teen from nevada"""
message_history.append({"role": "user", "content": user_message})

system_prompt = f"""You are a helpful product support chatbot for Intuit Quickbooks. You have access to the chat history with the user and you should provide output in strict JSON mode according to the following schema:

{combined_result}
"""

if planner_mode in ['fast','fast_listen_override']:
    model_name = 'gemini-2.0-flash'
elif planner_mode in ['smart']:
    model_name = 'gemini-2.0-pro-exp-02-05'
else:
    print("Invalid planner mode")


out_str = hlp.generate_flexible_structure(
    user_message = user_message,
    message_history = message_history,
    google_api_key = hlp.google_api_key,
    system_prompt = system_prompt,
    model_name  = model_name
)

validated_structure_out = hlp.validate_and_restructure(out_str[1], tools)

#message_history.append({"role": "assistant", "content": bot_reply})

In [8]:
validated_structure_out['Payroll']
validated_structure_out

{'TaxToolz': {'parameters': {'region2': 'Nevada', 'age': 16},
  'relevance_score': 10,
  'required_fields_completed': 30,
  'validation': {'relevance_score_error': False,
   'required_fields_completed_error': False,
   'required_params_error': False,
   'missing_required_params': [],
   'missing_or_invalid_optional_params': [],
   'fields_found': ['relevance_score',
    'required_fields_completed',
    'region2',
    'age'],
   'error_string': ''}},
 'Payroll': {'parameters': {'region2': 'Nevada'},
  'relevance_score': 95,
  'required_fields_completed': 100,
  'validation': {'relevance_score_error': False,
   'required_fields_completed_error': False,
   'required_params_error': False,
   'missing_required_params': [],
   'missing_or_invalid_optional_params': [],
   'fields_found': ['relevance_score', 'required_fields_completed', 'region2'],
   'error_string': ''}}}

In [9]:
tools

[{'name': 'TaxToolz',
  'description': 'Handles tax-related queries',
  'priority': 10,
  'display_mode': 'as-is',
  'index_name': 'one',
  'top_k': 4,
  'reranker': 'top3',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"},{"name":"age","type":"integer","required":true,"description":"User exact age"}]',
  'user_fields_mapping': {'age': '23'},
  'disambiguation_level': 4,
  'can_be_overridden_when_sticky': False},
 {'name': 'Payroll',
  'description': 'Call this tool for payroll questions',
  'priority': 10,
  'display_mode': 'as-is',
  'index_name': 'two',
  'top_k': 12,
  'reranker': 'top1',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"}]',
  'user_fields_mapping': {},
  'disambiguation_level': 10,
  'can_be_overridden_when_sticky': True}]

In [122]:
v.get("relevance_score")

95

In [10]:
import json

successful_tools = []
need_disambig_tools = []

for k,v in validated_structure_out.items():
    if v.get("relevance_score") >= planner_config['relevance_threshold']:
        for t in tools:
            if t['name'] == k: 
                
                payload = {'name':k,'description':t['description'],'display_mode':t['display_mode'],'can_be_overridden_when_sticky':t['can_be_overridden_when_sticky'],'priority':t['priority']}

                if v.get('required_fields_completed') >= t['disambiguation_level']*10 and len(v.get('validation')['missing_required_params'])==0:
                    # all required fields are completed and disambiguation level is met
                    
                    payload['disambiguation_fields'] = []
                    successful_tools.append(payload)
                else: # disambiguation needed

                    parameters_dict = json.loads(t['parameters_json'])

                    if v.get('required_fields_completed') < t['disambiguation_level']*10: # add all fields to disambituation because threshold not met
                        
                        print(t['disambiguation_level']*10)
                        print(v.get('required_fields_completed'))

                        disambiguation_fields = []
                        for p in parameters_dict:
                            disambiguation_fields.append({p['name']:p['description']})
                    else: # threshold met, but some fields are missing so add them to disambiguation
                        disambiguation_fields = []
                        
                        for p in parameters_dict:
                            if p['name'] in v.get('validation')['missing_required_params']:
                                disambiguation_fields.append({p['name']:p['description']})
                                

                    payload['disambiguation_fields'] = disambiguation_fields
                    need_disambig_tools.append(payload)
                


        
def get_lowest_priority_items(items_list):
    if not items_list:
        return []
    
    # Find the minimum priority value
    min_priority = min(item['priority'] for item in items_list)
    
    # Filter the list to only include items with the minimum priority
    lowest_priority_items = [item for item in items_list if item['priority'] == min_priority]
    
    return lowest_priority_items


successful_tools_by_priority = get_lowest_priority_items(successful_tools)
need_disambig_tools_by_priority = get_lowest_priority_items(need_disambig_tools)
print(successful_tools_by_priority)
print("$$$$$$$$$$$$$$$$")
print(need_disambig_tools_by_priority)

# successful_tools,need_disambig_tools

# turn off planner disambiguation setting
# 

40
30
[{'name': 'Payroll', 'description': 'Call this tool for payroll questions', 'display_mode': 'as-is', 'can_be_overridden_when_sticky': True, 'priority': 10, 'disambiguation_fields': []}]
$$$$$$$$$$$$$$$$
[{'name': 'TaxToolz', 'description': 'Handles tax-related queries', 'display_mode': 'as-is', 'can_be_overridden_when_sticky': False, 'priority': 10, 'disambiguation_fields': [{'region2': 'User region'}, {'age': 'User exact age'}]}]


In [ ]:
#log all successful tools, even if disambiguation needed, even if lower priority
#execute all successful tools and after execution
# found successful solution with resply?
    # send all summarizable to be summarized
    # append all non summarizable
# if disambiguation needed, send disambig to summarization / w tool descriptions
# if nothing succeded, then send default response

# if no execution needed but disambiguation needed, summarize w/ disambig
# if no execution needed, have a default response
# if execution needed, find content under tool index (in sequence, if multiple tools) and filter using user parameters

# find content with top k for tool and rerank

# summarize if summarizable, even with multiple tools
# disambiguate against returened queries if below secondary disambiguation threshold and make tool sticky
# concatinate if not summarizable

In [79]:
ret.diagnose_retrieval_system()

Current working directory: c:\Users\dusan\OneDrive\Desktop\pp\plannerv2
Index directory: c:\Users\dusan\OneDrive\Desktop\pp\plannerv2\config\faiss_indexes
  - Exists: True
  - Is directory: True
  - Readable: True
  - Writable: True
Content directory: c:\Users\dusan\OneDrive\Desktop\pp\plannerv2\config\faiss_content
  - Exists: True
  - Is directory: True
  - Readable: True
  - Writable: True
Index files: ['.index', 'one.index', 'payrollIndex.index', 'three!.index', 'three.index', 'two.index']
Index: .index
  - Size: 7725 bytes
  - Modified: 1742107186.3285563
  - Readable: True
  - Content file exists: Yes
  - Content size: 774 bytes
  - Content modified: 1742107186.3295543
  - Content items count: 1
  - Content mapping count: 5
  - Content titles:
    - Adding Payroll Features
Index: one.index
  - Size: 7725 bytes
  - Modified: 1742187608.8000574
  - Readable: True
  - Content file exists: Yes
  - Content size: 791 bytes
  - Content modified: 1742187608.8010564
  - Content items coun

'Diagnosis complete'

In [62]:
content_config

{'items': [{'title': 'Adding Payroll Features',
   'body': "To add payroll features to your QuickBooks account, follow these steps:\r\n1. Navigate to Employees > Payroll Setup\r\n2. Click on 'Set up payroll'\r\n3. Choose your payroll plan (Basic, Enhanced, or Full Service)\r\n4. Enter your business information\r\n5. Add employee details\r\n6. Set up payroll taxes\r\n7. Complete payroll setup by following the on-screen instructions",
   'index_name': 'three',
   'query_strings': ['how do I add payrolls?',
    'setting up payroll',
    'payroll setup instructions',
    'how to enable payroll'],
   'user_fields_mapping': {'region32': ['US', 'Canada'], 'age': ['12', '23']},
   'tags': ['SDR', 'Bullets']},
  {'title': 'Tax Form Filing Deadlines',
   'body': 'Here are the important tax filing deadlines:\r\n- W-2 Forms: Must be provided to employees by January 31\r\n- Quarterly Payroll Taxes (Form 941): Due at the end of the month following each quarter\r\n- Annual Federal Unemployment (FUTA)

In [47]:

%load_ext autoreload
%autoreload 2
import retrieval as ret
import helper as hlp

# Load your configurations
configs = hlp.load_all_configs()
content_config = configs["content"]

# Create an indexer
indexer = ret.ContentIndexer()

# Build all indexes (you may want to comment this out after first run to save time)
ret.build_all_indexes(content_config, indexer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating/updating index 'three' with 4 items
Index created successfully with 21 vectors (including query strings)
Built 1 indexes: three


In [58]:
ret.build_all_indexes(content_config, indexer)

Creating/updating index 'three' with 4 items
Index created successfully with 21 vectors (including query strings)
Built 1 indexes: three


In [5]:

%load_ext autoreload
%autoreload 2
import retrieval as ret
import helper as hlp

from retrieval import retrieve_filtered_content

# Get content that matches US region and age 12
results = retrieve_filtered_content(
    index_name="one",
    query="tax filing globally",
    user_fields={"region32": "US", "age": "23"},
    top_k=3,
    similarity_threshold=.1
)

# This should return content that has both region32=US and age=12 in its user_fields_mapping
for item in results:
    print(f"Title: {item['title']}")
    print(f"Similarity: {item['similarity']:.2f}")
    print(f"Body: {item['body'][:100]}...")
    print("---")

# # Example with no match (South Africa is not in the list)
# no_match_results = retrieve_filtered_content(
#     index_name="one",
#     query="tax filing deadlines",
#     user_fields={"region32": "South Africa", "age": "12"},
#     top_k=3,
#     similarity_threshold=0.1
# )

# print(f"Number of results with South Africa filter: {len(no_match_results)}")  # Should be 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using index store path: C:/Users/dusan/OneDrive/Desktop/pp/plannerv2/config/faiss_indexes
Using content store path: C:/Users/dusan/OneDrive/Desktop/pp/plannerv2/config/faiss_content
Title: Adding Payroll Featuresz
Similarity: 0.98
Body: To add payroll features to your QuickBooks account, follow these steps:
1. Navigate to Employees > ...
---


In [77]:
results

[{'title': 'Setting Up International Payments',
  'body': "To set up international payments in QuickBooks:\r\n1. Go to Banking > Bank Feeds > Set Up Bank Feed\r\n2. Select your bank or 'Connect to a different bank'\r\n3. For international payments, select the currency\r\n4. Enter your bank account details\r\n5. Set up the payment method\r\n6. Test the connection with a small transaction\r\n7. Start using international payments",
  'index_name': 'three',
  'query_strings': ['how to make international payments',
   'global payment setup',
   'paying vendors internationally',
   'foreign currency payments'],
  'user_fields_mapping': {'region32': ['US', 'Canada', 'Serbia', 'Croatia'],
   'age': ['12', '23']},
  'tags': [],
  'similarity': np.float32(0.9886756)},
 {'title': 'Tax Form Filing Deadlines',
  'body': 'Here are the important tax filing deadlines:\r\n- W-2 Forms: Must be provided to employees by January 31\r\n- Quarterly Payroll Taxes (Form 941): Due at the end of the month follow

In [26]:
import rerankers as rr
# Apply different rerankers
top1_results = rr.top1_reranker(results)
top3_results = rr.top3_reranker(results)
sdr_boosted_results = rr.uprank_sdr_reranker(results)

# Display top1 result
print("Top 1 Result:")
if top1_results:
    print(f"Title: {top1_results[0]['title']}")
    print(f"Similarity: {top1_results[0]['similarity']:.4f}")
else:
    print("No results found")

# Display top3 results
print("\nTop 3 Results:")
for i, item in enumerate(top3_results, 1):
    print(f"{i}. {item['title']} (Similarity: {item['similarity']:.4f})")

# Display SDR-boosted result
print("\nSDR-Boosted Result:")
if sdr_boosted_results:
    item = sdr_boosted_results[0]
    boosted = item.get('boosted', False)
    print(f"Title: {item['title']}")
    print(f"Similarity: {item['similarity']:.4f}{' (boosted)' if boosted else ''}")
else:
    print("No results found")

Top 1 Result:
Title: Setting Up International Payments
Similarity: 0.9887

Top 3 Results:
1. Setting Up International Payments (Similarity: 0.9887)
2. Setting Up International Payments 2 (Similarity: 0.9873)

SDR-Boosted Result:
Title: Setting Up International Payments 2
Similarity: 0.9898 (boosted)


In [53]:
import os

# Get file info
def get_file_info(path):
    if os.path.exists(path):
        return {
            'size': os.path.getsize(path),
            'modified': os.path.getmtime(path)
        }
    return None

# Get info for the index files
index_path = "config/faiss_indexes/three.index"
content_path = "config/faiss_content/three.pkl"

# Print file info
index_info = get_file_info(index_path)
content_info = get_file_info(content_path)

print(f"Index file size: {index_info['size']} bytes, modified: {index_info['modified']}")
print(f"Content file size: {content_info['size']} bytes, modified: {content_info['modified']}")

Index file size: 32301 bytes, modified: 1742108782.837486
Content file size: 2778 bytes, modified: 1742108782.837486


In [49]:
import os

# Get file info
def get_file_info(path):
    if os.path.exists(path):
        return {
            'size': os.path.getsize(path),
            'modified': os.path.getmtime(path)
        }
    return None

# Get info for the index files
index_path = "config/faiss_indexes/three.index"
content_path = "config/faiss_content/three.pkl"

# Print file info
index_info = get_file_info(index_path)
content_info = get_file_info(content_path)

print(f"Index file size: {index_info['size']} bytes, modified: {index_info['modified']}")
print(f"Content file size: {content_info['size']} bytes, modified: {content_info['modified']}")

Index file size: 32301 bytes, modified: 1742108782.837486
Content file size: 2778 bytes, modified: 1742108782.837486


In [56]:
import pickle

# Load the content directly
with open("config/faiss_content/three.pkl", 'rb') as f:
    data = pickle.load(f)
    content_items = data['content_items']

# Print the titles and some details
for item in content_items:
    print(f"Title: {item['title']}")
    print(f"Index name: {item['index_name']}")
    print(f"User fields: {item.get('user_fields_mapping', {})}")
    print("---")

Title: Adding Payroll Features
Index name: three
User fields: {'region32': ['US', 'Canada'], 'age': ['12', '23']}
---
Title: Tax Form Filing Deadlines
Index name: three
User fields: {'region32': ['US', 'Canada'], 'age': ['12', '23']}
---
Title: Setting Up International Payments
Index name: three
User fields: {'region32': ['US', 'Canada', 'Serbia', 'Croatia'], 'age': ['12', '23']}
---
Title: Setting Up International Payments 2
Index name: three
User fields: {'region32': ['US', 'Canada', 'Serbia', 'Croatia'], 'age': ['12', '23']}
---


In [57]:
content_items

[{'title': 'Adding Payroll Features',
  'body': "To add payroll features to your QuickBooks account, follow these steps:\r\n1. Navigate to Employees > Payroll Setup\r\n2. Click on 'Set up payroll'\r\n3. Choose your payroll plan (Basic, Enhanced, or Full Service)\r\n4. Enter your business information\r\n5. Add employee details\r\n6. Set up payroll taxes\r\n7. Complete payroll setup by following the on-screen instructions",
  'index_name': 'three',
  'query_strings': ['how do I add payrolls?',
   'setting up payroll',
   'payroll setup instructions',
   'how to enable payroll'],
  'user_fields_mapping': {'region32': ['US', 'Canada'], 'age': ['12', '23']},
  'tags': ['SDR', 'Bullets']},
 {'title': 'Tax Form Filing Deadlines',
  'body': 'Here are the important tax filing deadlines:\r\n- W-2 Forms: Must be provided to employees by January 31\r\n- Quarterly Payroll Taxes (Form 941): Due at the end of the month following each quarter\r\n- Annual Federal Unemployment (FUTA) Tax Return (Form 94